# 트리의 앙상블
##### 머신러닝(정형 데이터사용. 비정형 사용 힘듬), 딥러닝(비정형 데이터 사용가능)
##### 정형 데이터에 가장 우수한 성과를 내는 알고리즘 : 결정트리 기반의 앙상블 학습.

## 랜덤포레스트
##### 결정 트리를 랜덤하게 여러개 만들어 숲을 형성. 랜덤으로 만들기 때문에 트리의 성능을 낮춰 과대 적합 방지
##### 부트스트랩 샘플(n개의 샘플에 n번뽑아옴), 중복허용 추출, 훈련세트와 테스트 세트 크기가 같도록 추출, 결정 트리는 100개 생성
##### 분류 : 각 100개 결정 트리의 최종 리프 클래스별 확률(분류 개수비율)의 평균으로 결정
##### 회귀 : 각 결정 트리의 예측확률(분류 개수 비율)의 합/트리의 계수 -> 예측 확률의 평균

In [1]:
#노드마다 랜덤하게 선택한 특성을 사용해 과대적합 방지. 최적의 분할이 되지 않도록 방해한다. 그 뒤 각 결정트리는 최선의 분할 선택 작업을 수행
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate   #교차검증 수행
from sklearn.ensemble import RandomForestClassifier   #앙상블 분류 클래스

rf = RandomForestClassifier(n_jobs=-1, random_state=42)  #기본 100개 트리, 모든코어 사용
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) #return_train_score 기본값은 검증 세트 점수만 반환, true로 훈련세트도 본다

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #이름 테스트지만 검증세트의 점수이다.

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)#rf모델의 특성 중요도 계산의 위한 fit
print(rf.feature_importances_) #결정트리보다 좀더 완만한 결과(과대적합을 줄이고 일반화된 성능화)

[0.23167441 0.50039841 0.26792718]


In [4]:
### RandomForestClassifier의 자체적 모델평가기능 oob_score
### 부트스트랩에 포함되지 않고 남은 샘플을 사용해 트리 재평가.
#누락 샘플들 합쳐 검증세트로서의 역할 수행
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) 

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리
##### 부트스트랩핑 대신 전체 훈련 세트 데이터 모두 사용(즉, 동일한 n개의 샘플 모두를 훈련과 테스트 세트에 사용)
##### 노드 분할 시, 성능 억제를 위해 최적 분할 대신 무작위 분할 사용
##### 분할을 위한 불순도 개념 없음. 무작위 랜덤 분할->빠른 학습속도 제공
##### 무작위 랜덤 분할로 과대적합 방지, 검증 세트 점수 높이는 효과. 대신 무작위성이 커 100개 이상의 트리를 권장

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅
###### 그레디언트는 손실함수의 최저점을 찾는 오차줄이기 기법(분류는 로지스틱손실함수, 회귀는 평균 제곱 오차)
##### 앙상블 알고리즘 가운데 일반적으로 가장 성능이 좋다
##### 손실이 낮아지도록 결정트리를 계속추가하는 방법. 결정트리의 결과값으로 분류/회귀 결과 예측
##### 이때 예측값을 오차 함수에 대입하여 오차 확인. 오차가 줄어드는 방향으로 질문을 재구성한 신규 트리를 앙상블에 추가.
##### ->현 트리의 결과값을 분석해 오차가 낮아지는 방향으로 새로운 트리를 순차적으로 추가.
##### 깊이가 얕은 결정트리를 계속 추가하여 손실(오차) 보완. 깊이가 3인 트리 100개사용
##### 학습률 매개변수로 학습 속도 조정가능. 깊이가 얕아 과대적합에 강하며 높은 성능을 보인다.
##### 하지만 트리를 순서대로 추가하기 때문에 병렬성이 결여되어 속도가 느리다.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #과대적합 거의 발생안함

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)#트리의 개수 500개. 학습률로 이동속도 조정
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #500개여도 과대적합 잘 억제됨

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅
##### 사이킷런에서 제공하는 다른 그레이디언트 부스팅 기법
##### 훈련데이터샘플을 256개의 구간으로 구분. 1/256개의 데이터로 깊이 3인 트리를 계속 추가하여 최적화 속도가 빠름
##### 오차를 줄이기 위한 최적 분할용 질문 선택을 보다 더 빠르게 수행

In [10]:
from sklearn.experimental import enable_hist_gradient_boosting # 정식은 아니고 실험적 패키지
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #과대적합 억제하면서 기본 그레이디언트 부스팅보다 좀더 좋은 성능.

0.9321723946453317 0.8801241948619236


/Users/simsanghyeog/opt/anaconda3/envs/tfconda/lib/python3.7/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [11]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [12]:
hgb.score(test_input, test_target)

0.8723076923076923

In [15]:
#permutaion importance
#히스토그램 기반 gb는 기존 앙상블에 존재한 특성 중요도 속성이 없다. 이를 위해 permutaion_importnace()사용
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state=42, n_jobs=-1) #10번 반복
print(result.importances_mean) #분류기 때문에 합이 1이 아니고 정확도의 차이로 계산. 당도를 섞으니 정확도가 23% 감소함을 의미한다.

[0.08876275 0.23438522 0.08027708]


In [16]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [17]:
hgb.score(test_input, test_target) #87% 정확도

0.8723076923076923

#### XGBoost

In [18]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/Users/simsanghyeog/opt/anaconda3/envs/tfconda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/simsanghyeog/opt/anaconda3/envs/tfconda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/simsanghyeog/opt/anac

[16:12:42] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:12:42] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:12:42] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

#### LightGBM
##### 마이크로 소프트의 히스토그램 기반 gb라이브러리

In [14]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


앙상블 학습  
더 좋은 예측 결과를 만들기 위해 여러개의 모델을 훈련하는 머신러닝 알고리즘  
정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘. 대부분 결정 트리를 기반으로 한다.  
  
랜덤 포레스트  
대표적 결정 트리 기반의 앙상블 학습 방법. 부트스트랩 샘플을 사용  
랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징  
랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련세트에 과대적합되는 것을 막아준다.  
검증세트와 테스트세트에서 안정적인 성능을 얻을 수 있다.  
  
부트스트랩 샘플: 무작위로 중복을 허용해서 선택한 n개의 데이터를 선택하는 과정으로 샘플링하여 분류한 데이터  
oob샘플 : 부트스트랩 샘플에 포함되지 않은 샘플